In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

In [2]:
#paths required
datafolder_p = r'D:\18-DS\github\SDSHL\data\processed'
datafolder_e = r'D:\18-DS\github\SDSHL\data\external'
datafolder_i = r'D:\18-DS\github\SDSHL\data\internal'
modelfolder  = r'D:\18-DS\data\models'
resultsfolder = r'D:\18-DS\github\SDSHL\data\results'

file_train = datafolder_p + r'\2-train.csv'
file_test = datafolder_p + r'\2-test.csv'
file_org  = datafolder_p + r'\1-Hinglish_SarcasmCSV.csv'

#uncomment variable below to calculate metrics to include prediction of combined embedding
#This file is created after selecting best embedding and combining that with lexical features
#That is possible only when we run all the models of 09.3-Calssification-All-in-One notebook & 
#combined the prediction results of fasttext, bert, cnn
#prediction_file = r'\model_predictions_combined.csv'

In [3]:
model_with_data_list=['Normal','RandomOverSampler','Smote','AdaSyn']
option=0

# Combine Predictions of All the Models

In [13]:
def combine_all_prediction(pred_mainfilename):
    #Reading predictions of classifcal models
    df_prediction  = pd.read_csv( resultsfolder + pred_mainfilename ,index_col="ID" )

    #FastText Prediction using Pretrained Model with Task Finetuning
    df_prediction_fasttext = pd.read_csv( resultsfolder + r'\model_predictions_fasttext.csv', index_col="ID")

    #BERT Prediction using Pretrained Model with Task Finetuning
    df_prediction_BERT = pd.read_csv( resultsfolder + r'\model_predictions_BERT.csv', index_col="ID")

    #Neural Network Prediction
    df_prediction_CNN      = pd.read_csv( resultsfolder + r'\model_predictions_NN.csv', index_col="ID")

    #Combine All Predictions
    df_prediction = pd.concat( [df_prediction,df_prediction_fasttext, df_prediction_BERT, \
                                df_prediction_CNN], axis=1, ignore_index=False )

    #df_prediction = df_prediction.drop(columns=['label'])
    #df_prediction['label'] = label

    #Identify whether sentence is twitter or non-twitter. Also add label to the prediction
    df_org = pd.read_csv(file_org, sep='\t', index_col='ID')
    df_val = pd.read_csv(file_test, sep='\t', index_col='ID')
    idx_val = df_val.index

    #read only those records from original file which has ID equal to Validation dataset and get their label & Twitter flag
    df_prediction['label'] = df_org.loc[idx_val]['label']
    df_prediction['twitter'] = df_org.loc[idx_val]['twitter']
    return df_prediction

In [5]:
df_prediction = combine_all_prediction(r'\model_predictions_Normal.csv')

In [8]:
y_act          = df_prediction['label']

# Calulate Metrics using Predictions done in earlier step

In [6]:
threshold= 0.5

In [7]:
def calculate_metrics(y_act_, df_prediction_,):
    tot_models=df_prediction_.shape[1]-2
    #print ('Total Modles {tot_mod}'.format(tot_mod= tot_models))
    results = np.zeros( (tot_models,9) )

    final_label = lambda x: 1 if x>threshold else 0

    #y_act= df_prediction1.iloc[:, tot_models]
    model_name=[]
    for i in range(tot_models):

        #print (df_prediction_.columns[i], len(y_act_), len(df_prediction_.iloc[:,i]), i )

        model_name.append( df_prediction_.columns[i])

        #print (y_act_)
        results[i][0] = np.round( metrics.roc_auc_score(y_act_,df_prediction_.iloc[:,i]), 2)

        y_pre= df_prediction_.iloc[:,i].apply(final_label)

        results[i][1] = np.round( metrics.accuracy_score(y_act_,y_pre), 2)
        results[i][2] = np.round( metrics.recall_score(y_act_,y_pre), 2)
        results[i][3] = np.round( metrics.precision_score(y_act_,y_pre), 2)
        results[i][4] = np.round( metrics.f1_score(y_act_,y_pre), 2)
        
        cm= np.round( metrics.confusion_matrix(y_act_,y_pre), 2)
        results[i][5] = cm[0,0]
        results[i][6] = cm[0,1]
        results[i][7] = cm[1,0]
        results[i][8] = cm[1,1]
        #print (np.array(cm).reshape(-1))

        #results[i][4] = np.round( metrics.roc_auc_score(y_act,y_pre), 2)

    results1 = pd.DataFrame(results,columns=['AUC','Accuracy','Recall','Precision','F1','TN','FP','FN','TP'], index=model_name)

    return results1

In [9]:
results1 = calculate_metrics(y_act, df_prediction)

D:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
pd.set_option( "display.html.border" , 1 )
pd.set_option("display.max_rows",110)

In [11]:
print ("Number of Models {modnum}".format( modnum = len(results1)))

Number of Models 99


In [12]:
#Results for all Records
results1 = calculate_metrics(y_act, df_prediction)
results1
results1.sort_values("Accuracy",ascending=False)

D:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,AUC,Accuracy,Recall,Precision,F1,TN,FP,FN,TP
fastTextWiki_NB,0.80,0.76,0.78,0.75,0.76,74.0,26.0,22.0,78.0
fasttext_wiki_pretrained,0.81,0.76,0.71,0.79,0.75,81.0,19.0,29.0,71.0
IndicFT_XGB,0.79,0.74,0.70,0.76,0.73,78.0,22.0,30.0,70.0
mBERT_PytorchTT,0.80,0.74,0.69,0.76,0.72,78.0,22.0,31.0,69.0
fastTextWiki_SVC,0.81,0.74,0.67,0.79,0.72,82.0,18.0,33.0,67.0
IndicFT_ADB,0.79,0.74,0.72,0.76,0.74,77.0,23.0,28.0,72.0
fasttext_indicft_pretrained,0.81,0.74,0.71,0.76,0.74,78.0,22.0,29.0,71.0
IndicFT_LR,0.78,0.74,0.70,0.75,0.73,77.0,23.0,30.0,70.0
IndicFT_SVC,0.79,0.74,0.71,0.76,0.73,77.0,23.0,29.0,71.0
IndicFT_NB,0.77,0.74,0.70,0.76,0.73,78.0,22.0,30.0,70.0


# Combine Model Prediction

In [ ]:
# Here we know the best Embedding. 
# Combine this embedding with Lexical Features in "08.1-Classification-All-in-One-Shot" - At the end of file.
# Create a combined model and generate prediction results
# Check Here Whether Combined model is any better then the best model eariler.

In [14]:
#Load a file which has all prediction including of combined model
df_prediction = combine_all_prediction(r'\model_predictions_combined.csv')

In [15]:
#Split Twitter vs Non-Twitter records to to get metrics separately
y_act          = df_prediction['label']

df_prediction1 = df_prediction[df_prediction.twitter=='Y']
y_act1         = df_prediction[df_prediction.twitter=='Y']['label']

df_prediction2 = df_prediction[df_prediction.twitter=='N']
y_act2         = df_prediction[df_prediction.twitter=='N']['label']

In [16]:
results1 = calculate_metrics(y_act, df_prediction)

D:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
print ("Number of Models {modnum}".format( modnum = len(results1)))

Number of Models 109


In [18]:
#Results for all Records
results1 = calculate_metrics(y_act, df_prediction)
results1
results1.sort_values("Accuracy",ascending=False)

D:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,AUC,Accuracy,Recall,Precision,F1,TN,FP,FN,TP
fastTextWiki_NB,0.80,0.76,0.78,0.75,0.76,74.0,26.0,22.0,78.0
fasttext_wiki_pretrained,0.81,0.76,0.71,0.79,0.75,81.0,19.0,29.0,71.0
IndicFT_LR,0.78,0.74,0.70,0.75,0.73,77.0,23.0,30.0,70.0
mBERT_PytorchTT,0.80,0.74,0.69,0.76,0.72,78.0,22.0,31.0,69.0
Combined_NB,0.79,0.74,0.76,0.74,0.75,73.0,27.0,24.0,76.0
IndicFT_NB,0.77,0.74,0.70,0.76,0.73,78.0,22.0,30.0,70.0
fasttext_indicft_pretrained,0.81,0.74,0.71,0.76,0.74,78.0,22.0,29.0,71.0
fastTextWiki_SVC,0.81,0.74,0.67,0.79,0.72,82.0,18.0,33.0,67.0
IndicFT_SVC,0.79,0.74,0.71,0.76,0.73,77.0,23.0,29.0,71.0
IndicFT_XGB,0.79,0.74,0.70,0.76,0.73,78.0,22.0,30.0,70.0


In [19]:
#Save Metrics in a file for Reporting Purpose
fl= resultsfolder + r'\model_results_all.csv'
print ('All Results Prediction Results in file ', fl)
results1 = calculate_metrics(y_act, df_prediction)
results1.to_csv(fl)


f2= resultsfolder + r'\model_results_twitter.csv'
print ('Twitter Statement Prediction Results in file ', fl)
results1 = calculate_metrics(y_act1, df_prediction1)
results1.to_csv(f2)


f3= resultsfolder + r'\model_results_wo_twitter.csv'
print ('Normal Statements Prediction Results in file ',fl)
results1 = calculate_metrics(y_act2, df_prediction2)
results1.to_csv( f3)

All Results Prediction Results in file  D:\18-DS\github\SDSHL\data\results\model_results_all.csv


D:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Twitter Statement Prediction Results in file  D:\18-DS\github\SDSHL\data\results\model_results_all.csv


D:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Normal Statements Prediction Results in file  D:\18-DS\github\SDSHL\data\results\model_results_all.csv


D:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
